# Narreme Visualization

This second part of the code will focus on enphasizing the visualization and exploration of the different documents and topics we extracted from the previous notebook. By doing those we can understand the linking between each documents so that to create narrative connections between them.

This notebook is part of a master's thesis project in Digital Interaction Design at Politecnico di Milano, by Federico Denni.

In [1]:
from IPython.display import clear_output
!pip install pyvis networkx seaborn spacy
!python -m spacy download xx_sent_ud_sm
clear_output()

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy #nlp
from spacy import displacy #spacy visualizer library
import pyvis #interactive visualization
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt #plots
import seaborn as sns #make these plots nice

plt.style.use('ggplot')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/test-data/ready_data.csv
/kaggle/input/research/ready_data.csv


In [3]:
try:
    #Change this directory for other dataset
    df = pd.read_csv(r'/kaggle/input/research/ready_data.csv', encoding="utf-8")
    #eliminate compound scores
    
except FileNotFoundError:
      print("Error: file not found. Please upload the file or provide the correct path.")

In [4]:
df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000


---

Now that is loaded, lets proceed to analyze and clean the different elements

In [5]:
#let's split the docs_n in sentences
nlp = spacy.load("xx_sent_ud_sm")#translate all sentences in english before using this

# Define a function to tokenize text using spaCy
def tokenize(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [token.text for token in doc]
    return []

# Define a function to split text into sentences using spaCy
def split_sentences(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [sent.text for sent in doc.sents]
    return []

# Apply the sentence splitting function to the new columns
df['docs_1_sentences'] = df['docs_1'].apply(split_sentences)
df['docs_2_sentences'] = df['docs_2'].apply(split_sentences)
df['docs_3_sentences'] = df['docs_3'].apply(split_sentences)

df.to_csv(r'/kaggle/working/sentences.csv', index=False)

df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,[This is for everyone Just how you didn’t expe...,"[#, 4 is definitely one of the biggest critici...","[On adding symbols to the card, I disagree tha..."
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[I'm a game developer, and I realized that my ...",[How me and my friends play uno],[I like the card game of uno]
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,[You should be careful with the symbols (Golde...,"[Thanks for this list!, Soooo many games don't...","[Thanks for this list!, Soooo many games don't..."
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,[che bella collezione],[Che bella collezione 😍😍],[Che collezione 💪💪💪]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"[​@@TheHistoryGuyChannel ​Yah, as I recall, th...","[@@alymbouras it doesn't really matter, the im...","[@@alymbouras it doesn't really matter, the im..."
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,[It seems that every country has a card game o...,[One thing most cards are missing (which is we...,[@@StefanLopuszanski I have looked you up and ...


In [6]:
#lets now identify and extract the entities type in the sentences
nlp = spacy.load("en_core_web_sm")#translate all sentences in english before using this

df = pd.read_csv(r'/kaggle/working/sentences.csv', encoding="utf-8")
columns_to_extract = ['docs_1_sentences', 'docs_2_sentences', 'docs_3_sentences']

def get_entities_and_keywords(sentences, keywords):
    entities = []
    for sentence in sentences:
        doc = nlp(sentence)
        sentence_entities = [ent.label_ for ent in doc.ents] #if MISC, add word
        for keyword in keywords:
            if keyword in sentence:
                sentence_entities.append(keyword)
                
        entities.append(sentence_entities)
    return entities

# Apply the function to each column
df['doc_1_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_1_sentences']), eval(row['Keywords'])), axis=1)
df['doc_2_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_2_sentences']), eval(row['Keywords'])), axis=1)
df['doc_3_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_3_sentences']), eval(row['Keywords'])), axis=1)

df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences,doc_1_entity_list,doc_2_entity_list,doc_3_entity_list
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,['This is for everyone Just how you didn’t exp...,"['#', '4 is definitely one of the biggest crit...","['On adding symbols to the card, I disagree th...","[[PERSON], [PERSON], [], [], [TIME], [PERSON],...","[[], [CARDINAL, PERSON], [], [CARDINAL, text],...","[[symbols, text, card], [ORG, cards, text, car..."
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[""I'm a game developer, and I realized that my...",['How me and my friends play uno'],['I like the card game of uno'],"[[PERSON, playing, game, games, play, watching...","[[play, uno]]","[[game, like, uno]]"
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,['You should be careful with the symbols (Gold...,"['Thanks for this list!', ""Soooo many games do...","['Thanks for this list!', ""Soooo many games do...","[[symbols], [CARDINAL, rule], [symbols, games,...","[[], [CARDINAL, games, game, rule], [confusing...","[[], [CARDINAL, games, game, rule], [confusing..."
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,['che bella collezione'],['Che bella collezione 😍😍'],['Che collezione 💪💪💪'],"[[ORG, collezione]]",[[collezione]],[[collezione]]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"['\u200b@@TheHistoryGuyChannel \u200bYah, as I...","[""@@alymbouras it doesn't really matter, the i...","[""@@alymbouras it doesn't really matter, the i...","[[CARDINAL, played, play], [], [WORK_OF_ART, p...","[[], [ORG, games, game, cards]]","[[], [ORG, games, game, cards]]"
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,['It seems that every country has a card game ...,['One thing most cards are missing (which is w...,['@@StefanLopuszanski I have looked you up and...,"[[game, card], [ORG], [GPE, PERSON, game, play...","[[CARDINAL, playing, cards, card, play], [card...","[[], [PERCENT], [], [], [ORG], [ORG], [ORG], [..."


In [7]:
#transform all the list in a unique one and then eliminate whitespaces.
#filter out cardinals, percent
def extract_unique_words(list_of_lists):
    unique_words = set()
    for sublist in list_of_lists:
        unique_words.update(sublist)
    return [word for word in unique_words if word not in ['CARDINAL', 'PERCENT']]

# Apply the function to the specified columns
for column in ['doc_1_entity_list', 'doc_2_entity_list', 'doc_3_entity_list']:
    df[column] = df[column].apply(extract_unique_words)


df.to_csv(r'/kaggle/working/ent_list.csv', index=False)
df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences,doc_1_entity_list,doc_2_entity_list,doc_3_entity_list
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,['This is for everyone Just how you didn’t exp...,"['#', '4 is definitely one of the biggest crit...","['On adding symbols to the card, I disagree th...","[PERSON, TIME, WORK_OF_ART]","[text, card, LOC, game, PERSON, NORP, ORG, car...","[text, card, game, ORG, cards, symbols]"
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[""I'm a game developer, and I realized that my...",['How me and my friends play uno'],['I like the card game of uno'],"[watching, game, PERSON, play, playing, games]","[play, uno]","[uno, like, game]"
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,['You should be careful with the symbols (Gold...,"['Thanks for this list!', ""Soooo many games do...","['Thanks for this list!', ""Soooo many games do...","[game, rule, games, symbols]","[rules, game, rule, games, confusing, symbols]","[rules, game, rule, games, confusing, symbols]"
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,['che bella collezione'],['Che bella collezione 😍😍'],['Che collezione 💪💪💪'],"[collezione, ORG]",[collezione],[collezione]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"['\u200b@@TheHistoryGuyChannel \u200bYah, as I...","[""@@alymbouras it doesn't really matter, the i...","[""@@alymbouras it doesn't really matter, the i...","[played, poker, old, DATE, game, play, playing...","[games, ORG, cards, game]","[games, ORG, cards, game]"
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,['It seems that every country has a card game ...,['One thing most cards are missing (which is w...,['@@StefanLopuszanski I have looked you up and...,"[card, LOC, game, PERSON, play, ORG, cards, GPE]","[card, game, PERSON, play, playing, cards, gam...","[card, PERSON, play, ORG, cards]"


In [ ]:
import ast  # To safely evaluate string representations of lists
df = pd.read_csv(r'/kaggle/working/ent_list.csv', encoding="utf-8")

# Convert string representations of lists to actual Python lists
for col in ['doc_1_entity_list', 'doc_2_entity_list', 'doc_3_entity_list']:
    df[col] = df[col].apply(ast.literal_eval)

# Function to find matches and prepare source-target pairs with topic identifiers
def find_matches_with_topic(row):
    topic_id = row['Topic']  # Get the topic identifier
    doc_1_set = set(row['doc_1_entity_list'])
    doc_2_set = set(row['doc_2_entity_list'])
    doc_3_set = set(row['doc_3_entity_list'])

    # Compare each list with the others and collect matches
    matches = []

    # Compare doc_1 with doc_2 and doc_3
    for word in doc_1_set:
        if word in doc_2_set:
            matches.append({'source': word, 'target': word, 'source_doc': f'doc_1_topic_{topic_id}', 'target_doc': f'doc_2_topic_{topic_id}'})
        if word in doc_3_set:
            matches.append({'source': word, 'target': word, 'source_doc': f'doc_1_topic_{topic_id}', 'target_doc': f'doc_3_topic_{topic_id}'})

    # Compare doc_2 with doc_3
    for word in doc_2_set:
        if word in doc_3_set:
            matches.append({'source': word, 'target': word, 'source_doc': f'doc_2_topic_{topic_id}', 'target_doc': f'doc_3_topic_{topic_id}'})

    return matches

# Apply the function to each row and collect all matches in a flat list
all_matches = []
for _, row in df.iterrows():
    all_matches.extend(find_matches_with_topic(row))

# Create a DataFrame from the matches
matches_df = pd.DataFrame(all_matches)
matches_df.head(10)

In [10]:
#Let's now prepare the data for the network visualization
df = pd.read_csv(r'/kaggle/working/ent_list.csv', encoding="utf-8")

#now that we have a unique list of entities and not separate ones, lets add a new logic
"""
- First we will say to check between each other all the rows in all the 3 columns
- Second, if there are entities similar to each other they will be add in the target and source
"""

def create_source_target(df, entity_column, doc_number):
    source = []
    target = []
    source_membership = []
    target_membership = []
    for index, row in df.iterrows():
        entities = eval(row[entity_column])
        topic = row['Topic']
        for i in range(len(entities) - 1):
            for j in range(i + 1, len(entities)):
                source.append(str(entities[i]))
                target.append(str(entities[j]))
                source_membership.append(f"doc_{doc_number}_topic_{topic}")
                target_membership.append(f"doc_{doc_number}_topic_{topic}")
    return pd.Series([source, target, source_membership, target_membership])

#apply
df[['doc_1_source', 'doc_1_target', 'doc_1_source_membership', 'doc_1_target_membership']] = df.apply(lambda row: create_source_target(df, 'doc_1_entity_list', 1), axis=1)
df[['doc_2_source', 'doc_2_target', 'doc_2_source_membership', 'doc_2_target_membership']] = df.apply(lambda row: create_source_target(df, 'doc_2_entity_list', 2), axis=1)
df[['doc_3_source', 'doc_3_target', 'doc_3_source_membership', 'doc_3_target_membership']] = df.apply(lambda row: create_source_target(df, 'doc_3_entity_list', 3), axis=1)

In [11]:
source_target_df = pd.DataFrame({
    'source': df['doc_1_source'].explode().tolist() + df['doc_2_source'].explode().tolist() + df['doc_3_source'].explode().tolist(),
    'target': df['doc_1_target'].explode().tolist() + df['doc_2_target'].explode().tolist() + df['doc_3_target'].explode().tolist(),
    'source_membership': df['doc_1_source_membership'].explode().tolist() + df['doc_2_source_membership'].explode().tolist() + df['doc_3_source_membership'].explode().tolist(),
    'target_membership': df['doc_1_target_membership'].explode().tolist() + df['doc_2_target_membership'].explode().tolist() + df['doc_3_target_membership'].explode().tolist()
})

# Remove rows with empty source or target
source_target_df = source_target_df[(source_target_df['source'] != '[]') & (source_target_df['target'] != '[]')]

"""
If we have repeating a->b and b->a we first will switch the order of the columns to get all of them in the same direction
then we will clean if there is more than the same connection between each docs
"""

source_target_df.head(30)

,source,target,source_membership,target_membership
0,PERSON,TIME,doc_1_topic_0,doc_1_topic_0
1,PERSON,WORK_OF_ART,doc_1_topic_0,doc_1_topic_0
2,TIME,WORK_OF_ART,doc_1_topic_0,doc_1_topic_0
3,watching,game,doc_1_topic_1,doc_1_topic_1
4,watching,PERSON,doc_1_topic_1,doc_1_topic_1
5,watching,play,doc_1_topic_1,doc_1_topic_1
6,watching,playing,doc_1_topic_1,doc_1_topic_1
7,watching,games,doc_1_topic_1,doc_1_topic_1
8,game,PERSON,doc_1_topic_1,doc_1_topic_1
9,game,play,doc_1_topic_1,doc_1_topic_1


In [12]:
# Create a graph using networkx
G = nx.Graph()

"""
Here we need to group the different sources and targets in their own groups (determined by the doc_n_topic_n)
the connections will be externally to the doc_n_topic_n
"""

# Add nodes and edges for each document
for index, row in source_target_df.iterrows():
    G.add_node(row['source'], group=row['source_membership'])
    G.add_node(row['target'], group=row['target_membership'])
    G.add_edge(row['source'], row['target'])

"""
the labels needs to be only the doc_n_topic_n
colors instead will be the topic
also I need to think a way to simbolize the sentiment and compound value (maybe the transparency or shape)
"""

"""
Also to add better graphics
when I hover on the doc_n_topic_n cluster it would be interested to see the original docs text
then if I click it interesting to have it fixed, and see what is the keyword/entity highlighted on the network when I hover one of the nodes

"""
# Visualize the graph using pyvis
net = Network(notebook=True)
net.from_nx(G)

# Save the network visualization to an HTML file
net.save_graph('entity_network.html')
net.show('entity_network.html')

entity_network.html
